In [1]:
f = open("train.txt")

ll = []

for line in f.readlines():
    ll.append(line.split())
s = set([v for c in ll for v in c])
print(len(s))
f.close()


f = open("test-public.txt")
lltest = []
for line in f.readlines():
    lltest.append(line.split())
print(len(lltest))

4867136
2001


In [2]:
node_list = list(s)
    
startset = set([c[0] for c in ll])

line_number = {}
for c in ll:
    line_number[c[0]] = len(line_number)

In [3]:
m = {}
for c in ll:
    m[c[0]] = set(c[1:])
following = m

In [4]:
m2 = {}
for c in ll:
    for v in c[1:]:
        if v in m2:
            m2[v].add(c[0])
        else:
            m2[v] = set([c[0]])
followed = m2

In [5]:
m3 = {}
for c in ll:
    temp = 1/float(len(c))
    for v in c[1:]:
        if v in m3:
            m3[v][c[0]] = temp
        else:
            m3[v] = {}
            m3[v][c[0]] = temp
weightFollowed = m3

In [6]:
follewedLength= {}
for c in m3.keys():
    follewedLength[c] = sum([v*2 for v in m3[c].values()])**0.5

In [7]:
import random
train_pos_list = []
for l in ll:
    for i in range(1,min(2,len(l))):
        train_pos_list.append([1,l[0],l[i]])


train_neg_list = []
for l in ll:
    if len(train_neg_list) > 20000:
        break
    for i in range(10):
        temp = random.randint(0,len(node_list)-1)
        if node_list[temp] not in m[l[0]]:
            train_neg_list.append([1,l[0],node_list[temp]])
print(len(train_pos_list))
print(len(train_neg_list))

19570
20008


In [22]:
import networkx as nx
G = nx.Graph()
file = open('train.txt', mode='r')
train = dict()
for line in file:
    temp = line.split('\t')
    train[temp[0]] = temp[1:]

    
    
#positive sample
print('generate positive sample')
new_train = set()
for k, v in train.items():
    for values in v:
        new_train.add((int(k), int(values)))

print('add nodes')
G.add_edges_from(new_train)



generate positive sample
add nodes


In [106]:
train_pos_list_h = [[int(c[1]),int(c[2])] for c in train_pos_list]
train_neg_list_h = [[int(c[1]),int(c[2])] for c in train_neg_list]
test_list_h = [[int(c[1]),int(c[2])] for c in lltest[1:]]
RA_train_positive_set = nx.resource_allocation_index(G, train_pos_list_h)
RA_train_negative_set = nx.resource_allocation_index(G, train_neg_list_h)
RA_test_set = nx.resource_allocation_index(G, test_list_h)



In [107]:
temp = [p for u,v,p in RA_train_positive_set]
temp2 = [p for u,v,p in RA_train_negative_set]
temp3 = [p for u,v,p in RA_test_set]

train_pos_list_5 = []
for i in range(len(train_pos_list2)):
    train_pos_list_5.append(train_pos_list2[i]+[temp[i]])

train_neg_list_5 = []
for i in range(len(train_neg_list2)):
    train_neg_list_5.append(train_neg_list2[i]+[temp2[i]])
    
test_list_5 = []    
for i in range(len(test_list2)):
    test_list_5.append(test_list2[i]+[temp3[i]])



In [64]:
AA_train_positive_set = nx.adamic_adar_index(G, train_pos_list_h)
AA_train_negative_set = nx.adamic_adar_index(G, train_neg_list_h)
AA_test_set = nx.adamic_adar_index(G, test_list_h)

In [66]:
temp = [p for u,v,p in AA_train_positive_set]
temp2 = [p for u,v,p in AA_train_negative_set]
temp3 = [p for u,v,p in AA_test_set]

for i in range(len(train_pos_list2)):
    train_pos_list_5[i].append(temp[i])

for i in range(len(train_neg_list2)):
    train_neg_list_5[i].append(temp2[i])
    
for i in range(len(test_list2)):
    test_list_5[i].append(temp3[i])

In [110]:
JJ_train_positive_set = nx.jaccard_coefficient(G, train_pos_list_h)
JJ_train_negative_set = nx.jaccard_coefficient(G, train_neg_list_h)
JJ_test_set = nx.jaccard_coefficient(G, test_list_h)

In [111]:
temp = [p for u,v,p in JJ_train_positive_set]
temp2 = [p for u,v,p in JJ_train_negative_set]
temp3 = [p for u,v,p in JJ_test_set]

for i in range(len(train_pos_list2)):
    train_pos_list_5[i].append(temp[i])

for i in range(len(train_neg_list2)):
    train_neg_list_5[i].append(temp2[i])
    
for i in range(len(test_list2)):
    test_list_5[i].append(temp3[i])

In [112]:
train_pos_list_6 = [[c[0],c[1],c[2],c[3],c[4] for c in train_pos_list_5]
train_neg_list_6 = [[c[0],c[1],c[2],c[3],c[4] for c in train_neg_list_5]
test_list_6 = [[c[0],c[1],c[2],c[3],c[4] for c in test_list_5]

In [72]:
for c in train_pos_list_5:
    c.pop()

for c in train_neg_list_5:
    c.pop()
    
for c in test_list_5:
    c.pop()

In [8]:
def generateFeatures(dataList):
    resultList1 = []
    resultList2 = []
    for i,c in enumerate(dataList[1:]):
        if len(following[c[1]]) == 0:
            continue
        if i % 1000 == 0:
            print(i)
        temp3 = 0
        temp2 = 0
        for v in (set(m3[c[1]].keys())&set(m3[c[2]].keys())):
            temp2 += m3[c[1]][v] * m3[c[2]][v]
        temp3 += temp2 / follewedLength[c[1]] * follewedLength[c[2]]
        resultList1.append([temp3])
        temp3*=400

        tempscore = 1/float(len(following[c[1]]))
        temp2 = 0
        temp4 = 0
        for v1 in following[c[1]]:
            if v1 not in following:
                continue
            if len(following[v1]) ==0:
                continue
            tempscore2 = 1/float(len(following[v1]))
            for v2 in (following[v1] & set(m3[c[2]].keys())):
                temp2+=tempscore2*m3[c[2]][v2]
    #             if v2 not in following:
    #                 continue
    #             if len(following[v2]) ==0:
    #                 continue
    #             tempscore3 = 1/float(len(following[v2]))
    #             temp22 = 0
    #             for v3 in (following[v2] & set(m3[c[2]].keys())):
    #                 temp22+=tempscore3*m3[c[2]][v3]
    #             temp2+=tempscore2*temp22
        temp4+= temp2*tempscore
        temp3+=temp4
        resultList1[-1].append(temp4)

        temp4 = 0
        for v in (following[c[1]] & set(m3[c[2]].keys())):
            temp4+= tempscore*m3[c[2]][v]
        temp3+=temp4
        resultList1[-1].append(temp4)

        temp4 = 0
        if c[2] in following and c[1] in following[c[2]]:
            temp4+= 1/float(len(following[c[2]]))
        temp3+= temp4
        resultList1[-1].append(temp4)
        resultList2.append([temp3,i])
    return resultList1,resultList2

In [9]:
test_list2,temp = generateFeatures(lltest)
train_pos_list2,temp = generateFeatures(train_pos_list)
train_neg_list2,temp = generateFeatures(train_neg_list)


0
1000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000


In [114]:
print(train_pos_list_6[0])

[0.0, 0.00029578037648517907, 0.0004203028381854329, 0, 0.010297018215570252, 0.008064516129032258]


In [113]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
# LogisticRegression.fit()
clf = LinearRegression()
clf.fit(np.array(train_pos_list_6+train_neg_list_6),np.array([1]*len(train_pos_list_6)+[0]*len(train_neg_list_6)))

label = clf.predict(test_list_6)
print (label)

f3 = open("result5-1.txt","w")
f3.writelines("Id,Prediction\n")
for i,c in enumerate(label):
    f3.writelines(str(i+1)+","+str(c)+"\n")
f3.close()



[0.450525   0.48298405 0.450525   ... 0.45052475 0.450525   0.450525  ]


In [104]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
# LogisticRegression.fit()
clf = LogisticRegression()
clf.fit(np.array(train_pos_list_6+train_neg_list_6),np.array([1]*len(train_pos_list_6)+[0]*len(train_neg_list_6)))

label = clf.predict_proba(test_list_6)
print (label)

f3 = open("result5-1.txt","w")
f3.writelines("Id,Prediction\n")
for i,c in enumerate(label):
    f3.writelines(str(i+1)+","+str(c[1])+"\n")
f3.close()



[[0.55061904 0.44938096]
 [0.54447535 0.45552465]
 [0.55061905 0.44938095]
 ...
 [0.55061792 0.44938208]
 [0.55061905 0.44938095]
 [0.55061904 0.44938096]]


In [18]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
# LogisticRegression.fit()
clf = RandomForestClassifier()
clf.fit(np.array(train_pos_list_6+train_neg_list_6),np.array([1]*len(train_pos_list_6)+[0]*len(train_neg_list_6)))

label = clf.predict_proba(test_list_6)
print (label)

# RandomForestClassifier

[[0.6524032 0.3475968]
 [0.6524032 0.3475968]
 [0.6524032 0.3475968]
 ...
 [0.6524032 0.3475968]
 [0.6524032 0.3475968]
 [0.6524032 0.3475968]]


In [88]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(20005, 3), random_state=1, learning_rate=0.001, max_iter=1000, shuffle=True)
mlp.fit(np.array(train_pos_list_6+train_neg_list_6),np.array([1]*len(train_pos_list_6)+[0]*len(train_neg_list_6)))
label = mlp.predict_proba(test_list_6)


In [75]:
# from sklearn.neural_network import BernoulliRBM
# clf = BernoulliRBM()
# clf.fit(np.array(train_pos_list3+train_neg_list3),np.array([1]*len(train_pos_list3)+[0]*len(train_neg_list3)))

# label = clf.predict(test444)
# print len(label)


from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
# Models we will use
logistic = LogisticRegression()
rbm = BernoulliRBM(random_state=0, verbose=True)

classifier = Pipeline(steps=[('rbm', rbm), ('logistic', logistic)])

# #############################################################################
# Training

# Hyper-parameters. These were set by cross-validation,
# using a GridSearchCV. Here we are not performing cross-validation to
# save time.
rbm.learning_rate = 0.00001
rbm.n_iter = 200
# More components tend to give better prediction performance, but larger
# fitting time
rbm.n_components = 100
logistic.C = 6000.0

# Training RBM-Logistic Pipeline
classifier.fit(np.array(train_pos_list_5+train_neg_list_5),np.array([1]*len(train_pos_list_5)+[0]*len(train_neg_list_5)))

# Training Logistic regression
logistic_classifier = LogisticRegression(C=100.0)
logistic_classifier.fit(np.array(train_pos_list_5+train_neg_list_5),np.array([1]*len(train_pos_list_5)+[0]*len(train_neg_list_5)))

label = classifier.predict_proba(test_list_5)
print (label)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -2.48, time = 0.55s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -1.90, time = 0.88s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -1.53, time = 0.79s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -1.27, time = 0.79s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -1.09, time = 0.82s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -0.95, time = 0.78s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -0.85, time = 0.80s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -0.77, time = 0.84s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -0.71, time = 0.80s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -0.66, time = 0.84s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -0.61, time = 0.83s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -0.58, time = 0.79s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -0.55, time = 0.82s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -0.52, time = 0.80s
[BernoulliRBM] Iteration 15, pseudo-likelih

[BernoulliRBM] Iteration 120, pseudo-likelihood = -0.23, time = 0.79s
[BernoulliRBM] Iteration 121, pseudo-likelihood = -0.23, time = 0.82s
[BernoulliRBM] Iteration 122, pseudo-likelihood = -0.23, time = 0.79s
[BernoulliRBM] Iteration 123, pseudo-likelihood = -0.23, time = 0.79s
[BernoulliRBM] Iteration 124, pseudo-likelihood = -0.23, time = 0.80s
[BernoulliRBM] Iteration 125, pseudo-likelihood = -0.23, time = 0.80s
[BernoulliRBM] Iteration 126, pseudo-likelihood = -0.23, time = 0.83s
[BernoulliRBM] Iteration 127, pseudo-likelihood = -0.23, time = 0.84s
[BernoulliRBM] Iteration 128, pseudo-likelihood = -0.23, time = 0.87s
[BernoulliRBM] Iteration 129, pseudo-likelihood = -0.23, time = 0.81s
[BernoulliRBM] Iteration 130, pseudo-likelihood = -0.23, time = 0.82s
[BernoulliRBM] Iteration 131, pseudo-likelihood = -0.23, time = 0.80s
[BernoulliRBM] Iteration 132, pseudo-likelihood = -0.23, time = 0.83s
[BernoulliRBM] Iteration 133, pseudo-likelihood = -0.23, time = 0.86s
[BernoulliRBM] Itera

In [101]:
f3 = open("result5-1.txt","w")
f3.writelines("Id,Prediction\n")
for i,c in enumerate(label):
    f3.writelines(str(i+1)+","+str(c)+"\n")
f3.close()

In [24]:
def getPath(s,e):
    if e in following[s]:
        return 1
    if len(following[s]) == 0:
        return 1000
    tempset = set()
    pre = set([s])
    count = 0
    while len(pre)>0:
        count+=1
        tempset2 = set()
        for c in pre:
            if c not in following:
                continue
            if len(following[c]) == 0:
                continue
            tempset2 |= following[c]
        pre = tempset2 - tempset
        tempset |= tempset2
        if e in pre:
            return count
        if count > 4:
            break
    return 1000        